In [1]:
import pandas as pd
import plotly.graph_objects as go
import datetime
import numpy as np
import cProfile
import sys
import pickle
import random

from graph import TransportGraph
from forward_search import FCH

# Build transport graph

In [2]:
CITY = 'kuopio' #belfast, kuopio

## Optional 
### Add inverted road connections

In [3]:
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')

In [4]:
walk_connections['d_walk'].max()

1000

In [5]:
transport_connections = transport_connections[
    transport_connections['from_stop_I']!=transport_connections['to_stop_I']]

In [6]:
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

In [7]:
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

In [8]:
tg.edges_cnt, tg.nodes_cnt, tg.timetable_stats

(8891,
 549,
 {'min_size': 0,
  'mean_size': 3.2843324710381285,
  'std_size': 15.760627889293113,
  'max_size': 306})

# Build CH graph

### Calculate Sparce-CH

In [9]:
%%time
cProfile.run('ch_tg = tg.contraction_hierarchy(just_buses=True, max_walk_duration=1000)')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:01<00:00, 484.01it/s]

         19875792 function calls (16681592 primitive calls) in 9.101 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:100(acquire)
     10/3    0.000    0.000    0.019    0.006 <frozen importlib._bootstrap>:1022(_find_and_load)
     14/7    0.000    0.000    0.010    0.001 <frozen importlib._bootstrap>:1053(_handle_fromlist)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:125(release)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:165(__init__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:169(__enter__)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:173(__exit__)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:179(_get_module_lock)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1

In [10]:
ch_tg.edges_cnt, ch_tg.nodes_cnt, ch_tg.timetable_stats

(9672,
 549,
 {'min_size': 0,
  'mean_size': 6.403846153846154,
  'std_size': 19.320836541443924,
  'max_size': 306})

### Calculate Full-CH

In [11]:
%%time
cProfile.run('ch_tg_optimal = tg.contraction_hierarchy(just_buses=False)')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [01:29<00:00,  6.10it/s]


         118523158 function calls (115322650 primitive calls) in 99.118 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   99.118   99.118 <string>:1(<module>)
    15513    0.028    0.000    0.363    0.000 _collections_abc.py:957(pop)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:991(update)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:111(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:17(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:21(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:27(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:53(_commit_removals)
        3    0.000    0.000    0.000    0.000 _weakrefset.py:63(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:86(add)
       

In [10]:
%%time
cProfile.run('ch_tg_optimal = tg.contraction_hierarchy(just_buses=False)')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [05:48<00:00,  1.57it/s]

         430987112 function calls (428251135 primitive calls) in 356.385 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  356.385  356.385 <string>:1(<module>)
    18198    0.035    0.000    0.443    0.000 _collections_abc.py:957(pop)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:991(update)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:111(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:17(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:21(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:27(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:53(_commit_removals)
        3    0.000    0.000    0.000    0.000 _weakrefset.py:63(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:86(add)
      

In [12]:
ch_tg_optimal.edges_cnt, ch_tg_optimal.nodes_cnt, ch_tg_optimal.timetable_stats

(16049,
 549,
 {'min_size': 0,
  'mean_size': 62.92940370116518,
  'std_size': 63.75892593551478,
  'max_size': 542})

# Precalculate Geometrical containers for FCH

In [13]:
%%time
ch_tg_optimal.geometrical_container()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 549/549 [00:00<00:00, 3123.67it/s]

CPU times: user 176 ms, sys: 6.27 ms, total: 182 ms
Wall time: 183 ms


In [13]:
pickle.dump(ch_tg_optimal, open(F'{CITY}_original.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [14]:
pickle.dump(ch_tg, open(F'{CITY}_trunc.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [7]:
ch_tg = pickle.load(open(F'kuopio_trunc.pkl', 'rb'))
ch_tg_optimal = pickle.load(open(F'kuopio_original.pkl', 'rb'))

# Pathfinding

In [16]:
transport_connections['dep_time_ut'].min(), transport_connections['dep_time_ut'].max()

(1481511600, 1481585460)

In [17]:
test_pairs = pd.DataFrame([
    {'start_time': 1481514083, 'start_node': 330, 'end_node': 54},
    {'start_time': 1481519880, 'start_node': 345, 'end_node': 141},
    {'start_time': 1481518900, 'start_node': 1, 'end_node': 132},
    {'start_time': 1481514540, 'start_node': 116, 'end_node': 118},
    {'start_time': 1481519640, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 51},
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481571574, 'start_node': 146, 'end_node': 7}
])
for index, row in test_pairs.iterrows():
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60)
    print('Path duration', datetime.timedelta(seconds=int(path['arrival'] - row['start_time'])))
    print('Calculation duration', datetime.timedelta(seconds=int(path['duration'])))
    print()

Path duration 0:55:37
Calculation duration 0:01:08

Path duration 0:05:00
Calculation duration 0:00:07

Path duration 0:27:20
Calculation duration 0:01:06

Path duration 0:00:00
Calculation duration 0:00:00

Path duration 0:07:00
Calculation duration 0:00:11

Path duration 0:05:55
Calculation duration 0:00:05

Path duration 0:04:20
Calculation duration 0:00:04

Path duration 0:22:26
Calculation duration 0:00:29



In [18]:
test_pairs = pd.DataFrame([
    {'start_time': 1481514083, 'start_node': 330, 'end_node': 54},
    {'start_time': 1481519880, 'start_node': 345, 'end_node': 141},
    {'start_time': 1481518900, 'start_node': 1, 'end_node': 132},
    {'start_time': 1481514540, 'start_node': 116, 'end_node': 118},
    {'start_time': 1481519640, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 51},
    {'start_time': 1481515900, 'start_node': 1, 'end_node': 50}, 
    {'start_time': 1481571574, 'start_node': 146, 'end_node': 7}
])
for index, row in test_pairs.iterrows():
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, geometrical_containers=False)
    print('Path duration', datetime.timedelta(seconds=int(path['arrival'] - row['start_time'])))
    print('Calculation duration', datetime.timedelta(seconds=int(path['duration'])))
    print()

Path duration 0:55:37
Calculation duration 0:00:35

Path duration 0:05:00
Calculation duration 0:00:05

Path duration 0:27:20
Calculation duration 0:00:43

Path duration 0:00:00
Calculation duration 0:00:00

Path duration 0:07:00
Calculation duration 0:00:07

Path duration 0:05:55
Calculation duration 0:00:04

Path duration 0:04:20
Calculation duration 0:00:03

Path duration 0:22:26
Calculation duration 0:00:21



# Compare 2 solutions

In [14]:
N = 1_000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]
             })
new_duration = []
optimal_duration = []
for index, row in test_data.iterrows():
    
    # new search
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False,
                                    max_walk_duration=1000)
    if path['path']:
        new_duration.append(path['duration'])
    new_arrival = path['arrival']
    
    # optimal_duration
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True,
                                    max_walk_duration=1000)
    if path['path']:
        optimal_duration.append(path['duration'])
    optimal_arrival = path['arrival']
    
    # validate
    assert new_arrival == optimal_arrival

/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_31939/140951379.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)],
/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_31939/140951379.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]


AssertionError: 

In [61]:
pathfinding = FCH(graph=ch_tg,
                      start_time=1481556212,
                      start_node=269, 
                      end_node=249)
pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False, max_walk_duration=1000)

{'path': [269, 251, 252, 253, 250, 249],
 'routes': ['walk', '6', 'walk', '6', '6'],
 'arrival': 1481558580,
 'duration': 12}

In [53]:
pathfinding = FCH(graph=ch_tg,
                      start_time=1481557204,
                      start_node=251, 
                      end_node=249)
pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False, max_walk_duration=1000)

{'path': [251, 250, 249],
 'routes': ['walk', '6'],
 'arrival': 1481557380,
 'duration': 1}

In [47]:
path = [269, 251, 252, 253, 250, 249]
start_time = row['start_time']
for i, node in enumerate(path[:-1]):
    print(node, ch_tg.hierarchy[node], start_time)
    start_time = tg.graph[node][path[i+1]].arrival(start_time, max_walk_duration=1000)[0]


269 202 1481556212
251 397 1481557204
252 153 1481557638
253 140 1481557670
250 357 1481558061


In [71]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=1481556212,
                      start_node=269, 
                      end_node=250)
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True, max_walk_duration=1000)

{'path': [269, 251, 252, 253, 250],
 'routes': ['walk', '6', 'walk', '6'],
 'arrival': 1481558580,
 'duration': 20}

In [81]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=1481556212,
                      start_node=269, 
                      end_node=249)
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True, max_walk_duration=1000)

{'path': [269, 251, 252, 254, 256, 1131, 233, 257, 255, 253, 250, 249],
 'routes': ['walk', '6', '6', '6', 'walk', 'walk', '6', '6', '6', '6', '6'],
 'arrival': 1481559480,
 'duration': 47}

In [72]:
path = [269, 251, 252, 253, 250]
start_time = row['start_time']
for i, node in enumerate(path[:-1]):
    print(node, ch_tg_optimal.hierarchy[node], start_time)
    start_time = tg.graph[node][path[i+1]].arrival(start_time, max_walk_duration=1000)[0]


269 210 1481556212
251 399 1481557204
252 125 1481557638
253 140 1481557670


In [78]:
1481557204 - 1481556212

992

In [79]:
ch_tg_optimal.graph[251][249].arrival(1481557204, walk_duration=992, max_walk_duration=1000)

(1481560386, [251, 252, 250, 249], ['6', 'walk', 'walk'], 846)

In [80]:
path = [251, 252, 250, 249]
start_time = 1481557204
for i, node in enumerate(path[:-1]):
    print(node, ch_tg_optimal.hierarchy[node], start_time)
    start_time = tg.graph[node][path[i+1]].arrival(start_time, max_walk_duration=1000)[0]

251 399 1481557204
252 125 1481557638
250 253 1481558054


In [21]:
tg.graph()

1481556212

In [24]:
tg.graph[256][1131].arrival(1481556212)

(1481556334, [269, 268], ['walk'], 0)

In [26]:
tg.graph[268][244].arrival(1481556334)

(1481557329, [268, 244], ['walk'], 0)

In [27]:
1481557329 - 1481556212

1117

In [37]:
path = [249, 270, 246, 248, 266, 268, 113, 155, 107, 105, 109]
start_time = row['start_time']
for i, node in enumerate(path[:-1]):
    print(node, ch_tg.hierarchy[node], start_time)
    start_time = tg.graph[node][path[i+1]].arrival(start_time, max_walk_duration=1000)[0]


249 316 1481579006
270 436 1481579711
246 439 1481580184
248 342 1481580401
266 341 1481581363
268 269 1481581606
113 239 1481582115
155 212 1481582793
107 270 1481583004
105 297 1481583498


In [29]:
node, path[i+1]

(270, 270)

In [41]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True)

{'path': [249, 266, 268, 112, 109],
 'routes': ['walk', 'walk', 'walk', 'walk'],
 'arrival': 1481581283,
 'duration': 331}

In [42]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=268)
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True)

{'path': [249, 266, 268],
 'routes': ['walk', 'walk'],
 'arrival': 1481580234,
 'duration': 120}

In [48]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True,
                          max_walk_duration=1000)

{'path': [249, 270, 246, 248, 266, 268, 113, 155, 107, 105, 109],
 'routes': ['walk',
  'walk',
  '6',
  'walk',
  'walk',
  'walk',
  'walk',
  '41',
  '41',
  'walk'],
 'arrival': 1481584401,
 'duration': 20}

In [53]:
ch_tg_optimal.graph[249][246].walk.w

240

In [47]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=246)
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True,
                          max_walk_duration=1000)

{'path': [249, 246], 'routes': ['walk'], 'arrival': 1481579246, 'duration': 0}

In [29]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=43, 
                      end_node=36)
pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True,
                          max_walk_duration=1000)

{'path': [43, 143, 36, 148, 33, 36],
 'routes': ['walk', 'walk', '41', '41', 'walk'],
 'arrival': 1481579434,
 'duration': 51}

In [23]:
path = [249, 270, 246, 248, 266, 268, 113, 155, 107, 105, 109]
for i, node in enumerat(path):
    print(node, ch_tg_optimal.hierarchy[node], tg.graph[node[i], node[i+1]].arrival())

275 502
274 503
265 491
289 497
210 154
208 361
206 314
203 267
201 438
140 405
119 192
117 260
115 305
344 545
75 358
49 510
51 509
453 442


In [86]:
pathfinding.candidate_sequences[68]

[1335, 245, 62, 64, 68, 70, 72, 53, 54, 71, 69, 67, 68]

In [71]:
pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=70)
pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False)

{'path': [1335, 64, 70],
 'routes': ['walk', 'walk'],
 'arrival': 1481547860,
 'duration': 6}

In [72]:
1481547860 - row['start_time']

1444

In [68]:
pathfinding.candidate_sequences[68]

[1335, 64, 68, 70, 72, 71, 69, 68]

In [60]:
pathfinding_opt = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=53)
pathfinding_opt.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=True,
                                    max_walk_duration=1000)

{'path': [1335, 64, 68, 70, 72, 53],
 'routes': ['walk', '6', 'walk', '6', '6'],
 'arrival': 1481548140,
 'duration': 2}

In [61]:
pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=53)
pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False, max_walk_duration=1000)

{'path': [1335, 245, 62, 64, 68, 70, 72, 53],
 'routes': ['walk', '6', '6', '6', '6', '6', '6'],
 'arrival': 1481549340,
 'duration': 7}

In [57]:
ch_tg.graph[1335]

KeyError: 70

In [64]:
1481555669 - row['start_time']

974

In [65]:
row['start_time']

1481554695

In [16]:
np.mean(optimal_duration), np.median(optimal_duration), np.std(optimal_duration)

(13.254444444444445, 13.0, 7.552684013148514)

In [17]:
np.mean(new_duration), np.median(new_duration), np.std(new_duration)

(32.285555555555554, 35.0, 16.963084501424532)

### Withoout geometrical containers

In [18]:
N = 1_000
test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                           transport_connections['dep_time_ut'].max()) for i in range(N)],
             'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)], 
              'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]
             })
new_duration = []
optimal_duration = []
for index, row in test_data.iterrows():
    
    # new search
    pathfinding = FCH(graph=ch_tg,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=True, geometrical_containers=False)
    if path['path']:
        new_duration.append(path['duration'])
    new_arrival = path['arrival']
    
    # optimal_duration
    pathfinding = FCH(graph=ch_tg_optimal,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
    path = pathfinding.shortest_path(60, search_with_switching_graphs=False, geometrical_containers=False)
    if path['path']:
        optimal_duration.append(path['duration'])
    optimal_arrival = path['arrival']
    
    # validate
    assert new_arrival == optimal_arrival

/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_78265/2655839994.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)],
/var/folders/y3/4nz44bj52pz9p2mrm1s72mxc0000gn/T/ipykernel_78265/2655839994.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(ch_tg.nodes, 1)[0] for i in range(N)]


In [19]:
np.mean(optimal_duration), np.median(optimal_duration), np.std(optimal_duration)

(44.326966292134834, 42.0, 150.25644689868335)

In [20]:
np.mean(new_duration), np.median(new_duration), np.std(new_duration)

(32.63820224719101, 34.0, 16.91928960722136)

# Visualize path

In [8]:
%%time
pathfinding = FCH(graph=ch_tg,
                      start_time=1481519880,
                      start_node=345, 
                      end_node=141)
path = pathfinding.shortest_path(60)

CPU times: user 4.06 ms, sys: 1.18 ms, total: 5.24 ms
Wall time: 6.07 ms


In [9]:
nodes = pd.read_csv(F'data/{CITY}/network_nodes.csv', sep=';')
nodes_dict = nodes.set_index('stop_I').to_dict('index')

In [12]:
for station in path['path']:
    print(station, nodes_dict[station]['lat'], nodes_dict[station]['lon'], nodes_dict[station]['name'])

345 62.894633 27.645134 Canthia E
114 62.891878 27.637245 Snellmania E
116 62.88952 27.632591 Teknia E
118 62.888191 27.630635 Technopolis E
141 62.88333 27.61765 Neulamäentie 5 L


In [13]:
i = 0
k = 0
while i < len(path['routes']):
    x = path['routes'][i]
    if k == 0:
        i += 1
        if i == len(path['routes']):
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
        elif x != path['routes'][i]:
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
    else:
        i += 1
        if i < len(path['routes']):
            if x != path['routes'][i]:
                fig.add_trace(go.Scattermapbox(
                mode = "markers+lines",
                lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
                lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
                text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
                name = path['routes'][k],
                marker = {'size': 10}))
                k = i
        else:
            fig.add_trace(go.Scattermapbox(
            mode = "markers+lines",
            lon = [nodes_dict[path['path'][j]]['lon'] for j in range(k, i+1)],
            lat = [nodes_dict[path['path'][j]]['lat'] for j in range(k, i+1)],
            text = [(path['path'][j], nodes_dict[path['path'][j]]['name']) for j in range(k, i+1)],
            name = path['routes'][k],
            marker = {'size': 10}))
            k = i
        

fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': {'lon': nodes['lon'].iloc[0], 'lat': nodes['lat'].iloc[0]},
        'zoom': 9}, 
    showlegend=True)

fig.show()